# PACEUTILS

A package for computing values and data frames from the PACE-RI database. This guide can also be used to define certain calculations. Once the files are downloaded it can be installed using the command below (any updates by the data analyst to paceutils files will need be copied into the folder for the install to be updated);

`python -m pip install -e c:\path_to_paceutils_folder`

### Importing All Modules

In [ ]:
import paceutils as pu

### Importing A Single Module

In [ ]:
from paceutils import Enrollment

### Specifying the Database
The connection is opened and closed in each function.

In [ ]:
e = Enrollment() #default V:\Databases\PaceDashboard.db
# or
e = Enrollment("V:\\Databases\\PaceDashboard.db")

### Running Functions
You can run from the variable the class is initialized to or from the class itself.

In [ ]:
e.census_during_period(params = ('2019-01-01', '2019-01-01'))

In [ ]:
Enrollment().census_during_period(params = ('2019-01-01', '2019-01-01'))

### Common Definitions for Calculations (some of these are reiterated below)
**Member Months:** A participant is included in the member month count if they are enrolled as of the first of the month. This means a ppt enrolled on July 1st and disenrolled on September 22nd would have a member month count of 3.

**Net Enrollment:** The difference between the new enrollments on the first of a month and the disenrollments from the previous month.

**Census as of Date:** Count of ppts who have an enrollment date before or on the date and a disenrollment date on or after the date.

**Census Today:** Count of ppts with an enrollment date before or on today and no disenrollment date.

**Census Over Period:** Count of participants with a disenrollment date on or after the start date and an enrollment date before or on the end date of the period.

**Average Years Enrolled:** The difference between today or the ppts disenrollment date and their enrollment date divided by 365.25.

**Growth:** The difference between the census at the end of a period and the census at the start of a period divided by the census at the start of the period.

**Churn:** The quotient of disenrollments during a period and the census on the start date of the period.

**Adjusted Incident Count:** The count of an incidents minus any incidents attributed to ppts with an incident count 3 standard deviations above the average for ppts with an incident. I.e.: If the average number of falls per faller is 2 with a standard deviation of 0.5, then the falls of any ppts with a count above 2+(0.5\*3) = 3.5 would be excluded from the adjusted count.

**Living in the Community:** Count of ppts not in custodial care. Included respite, skilled, and assisted living ppts.

**Days vs LOS**: Length of Stay is only calculated for discharges and will cover the entire stay. Days is calculated for all admissions and if there is no discharge date, the end date of the period is used. When counting days in a month, the first of the month is used as the admission date for any admission with a date before the period.

## General Use Helpers

Functions for querying the database and parsing dates. ***All*** other classes inherit from the Helpers class.

In [ ]:
helpers = pu.Helpers()

### Single Value Query

Returns a single value from the database, uses .fetchone from the sqlite3 package. Takes a query and any necessary parameters (note: params can be left blank).

In [ ]:
helpers.single_value_query(q = "SELECT total FROM monthly_census WHERE month = ?",
                           params = ('2019-01-01',))

In [ ]:
helpers.single_value_query(q = "SELECT total FROM monthly_census WHERE month = '2019-01-01'")

### List of Tuple Query

Returns a list of tuples from the database, uses .fetchall from the sqlite3 package. Takes a query and any necessary parameters (note: params can be left blank).

In [ ]:
helpers.fetchall_query(q = "SELECT * FROM monthly_census WHERE MONTH >= ? LIMIT 5",
                       params = ('2019-01-01',))

In [ ]:
helpers.fetchall_query(q = "SELECT * FROM monthly_census WHERE MONTH >= '2019-01-01' LIMIT 5")

### DataFrame Query

Returns a pandas DataFrame from the database, uses pd.read_sql. Takes a query and any necessary parameters (note: params can be left blank).

In [ ]:
helpers.dataframe_query(q = "SELECT * FROM monthly_census WHERE MONTH >= ? LIMIT 5",
                        params = ('2019-01-01',))

In [ ]:
helpers.dataframe_query(q = "SELECT * FROM monthly_census WHERE MONTH >= '2019-01-01' LIMIT 5")

### Create a Time/Value DataFrame for Plotting

Takes an indicator calculating function and creates a pandas DataFrame of the monthly or quarterly value (based on freq parameter) for this indicator over the provided params. If no params are provided, it will create a DataFrame for the previous year. if the indicator need additional parameters other than date ranges, they can be provided in additional_func_args.

In [ ]:
helpers.loop_plot_df(indicator_func = e.census_on_end_date,
                     params = ('2019-01-01', '2019-09-30'),
                     freq="MS",
                     additional_func_args=None)

In [ ]:
helpers.loop_plot_df(indicator_func = e.census_on_end_date, freq="QS")

### Date-Related Helper Functions
The following require no parameters and return the dates based on today.

In [ ]:
print(helpers.month_to_date())
print(helpers.last_month())
print(helpers.last_three_months())
print(helpers.last_six_months())
print(helpers.last_year())
print(helpers.last_quarter())
print(helpers.quarter_to_date())

Last quarter can also the number and year of the previous quarter.

In [ ]:
helpers.last_quarter(return_q=True)

Returns the date for a given quarter and year.

In [ ]:
helpers.get_quarter_dates(q=2, yr=2019)

Takes a tuple of dates as the parameters and returns the dates for either the previous month or quarter.

In [ ]:
helpers.prev_month_dates(params=helpers.last_month())

In [ ]:
helpers.prev_quarter_dates(params=helpers.last_quarter())

## Enrollment

In [ ]:
enrollment = Enrollment()

### Census Today
Count of participants in enrollment file with a null disenrollment date.

In [ ]:
enrollment.census_today()

In [ ]:
params=("2019-01-01", "2019-03-31")

### Census During A Period
Count of ppts with an enrollment date prior to the end date and a disenrollment date that is null or after the start date.

In [ ]:
enrollment.census_during_period(params=params)

### Census at the End of a Period
Count of ppts with an enrollment date before the start date and a disenrollment date that is null or after the end date.

In [ ]:
enrollment.census_on_end_date(params=params)

### Member Months
Sum of the first of the month census for each month between the two param dates.

In [ ]:
enrollment.member_months(params=params)

### Disenrolled
Count of ppts with a disenrollment date between the two param dates.

In [ ]:
enrollment.disenrolled(params=params)

### Enrolled
Count of ppts with a enrollment date between the two param dates.

In [ ]:
enrollment.enrolled(params=params)

### Deaths
Count of ppts with a disenrollment date between the two param dates and a disenroll type of 'Deceased'.

In [ ]:
enrollment.deaths(params=params)

### Net Enrollment (total during period)
`enrolled` - `disenrolled` for the dates between the two param dates.

In [ ]:
enrollment.net_enrollment_during_period(params=params)

### Net Enrollment (for month)
`enrolled` for the dates between the two param dates - `disenrolled` for the month before the starting date of the params.

Difference between ppts enrolled on the first of a month and the disenrollments of the previous month. For a quarter this means we are adding up the number of new ppts on the first of each month in quarter and subtracting the number of disenrollment from the month before the quarter through the middle month of the quarter.

In [ ]:
enrollment.net_enrollment(params=params)

### Voluntary Disenrollments
Count of ppts with a disenrollment date between the two param dates and a disenroll type of 'Voluntary'.

In [ ]:
enrollment.voluntary_disenrolled(params=params)

### Percent of Disenrollments that are Voluntary
`voluntary_disenrolled` divided by `disenrolled` multiplied by 100.

In [ ]:
enrollment.voluntary_disenrolled_percent(params=params)

### Average Years Enrolled in PACE (for ppts enrolled in period)
Average of the difference between today's date or the ppt's disenrollment date and their enrollment date divided by 326.25.

In [ ]:
enrollment.avg_years_enrolled(params=params)

### Growth Rate
Census as of the first of the previous month from the given param dates minus the census as of the first of the month for the given params divided bu the census as of the first of the previous month from the given param dates multiplied by 100.

In [ ]:
enrollment.growth_rate(params=params)

### Churn Rate
`disenrolled` for the given params divided by the census as of the first date in params tuple.

In [ ]:
enrollment.churn_rate(params=params)

### Count of Ppts per Town (for ppts enrolled in period)
Count of distinct ppts grouped by town for ppts enrolled during the period. Returned as a pandas dataframe with columns `City/Town` and `Number of Ppts`.

In [ ]:
enrollment.enrollment_by_town_table(params=params).head()

### Mapping DataFrame
Creates two pandas dataframe with the columns `name`, `full_address`, `lat`, and `lon`, one for ppts enrolled during the period and one for those who are not.

In [ ]:
enrolled_df, disenrolled_df = enrollment.address_mapping_df()

print(enrolled_df.head())

### HPMS Related

### Dual Enrolled Enrollments
Count of ppts with an enrollment date during the period, medicare status of 1 and a medicaid status of 1.

In [ ]:
enrollment.dual_enrolled(params=params)

### Medicare Only Enrollments
Count of ppts with an enrollment date during the period, medicare status of 1 and a medicaid status of 0.

In [ ]:
enrollment.medicare_only_enrolled(params=params)

### medicaid Only Enrollments
Count of ppts with an enrollment date during the period, medicare status of 0 and a medicaid status of 1.

In [ ]:
enrollment.medicaid_only_enrolled(params=params)

### Private Pay Enrollments
Count of ppts with an enrollment date during the period, medicare status of 0 and a medicaid status of 0.

In [ ]:
enrollment.private_pay_enrolled(params=params)

### Dual Enrolled Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 1 and a medicaid status of 1.

In [ ]:
enrollment.dual_disenrolled(params=params)

### Medicare Only Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 1 and a medicaid status of 0.

In [ ]:
enrollment.medicare_only_disenrolled(params=params)

### medicaid Only Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 0 and a medicaid status of 1.

In [ ]:
enrollment.medicaid_only_disenrolled(params=params)

### Private Pay Disenrollments
Count of ppts with an disenrollment date during the period, medicare status of 0 and a medicaid status of 0.

In [ ]:
enrollment.private_pay_disenrolled(params=params)

### Marketing/Enrollment Referral Related

### Inquiries/Referrals
Count of rows from the referral table where the referral_date is between the param dates.

In [ ]:
enrollment.inquiries(params=params)

### Average Days to Enrollment
Average difference of the enrollment_effective date and the referral date for referrals with an enrollment_effective date between the param dates.

In [ ]:
enrollment.avg_days_to_enrollment(params=params)

### 180 Day Enrollment Conversion Rate for Referrals
Count of all referrals with a referral date between 180 days before the param start date and the param end date with a non-null enrollment effective date divided by the count of all referrals with a referral date between 180 days before the param start date and the param end date.

In [ ]:
enrollment.conversion_rate_180_days(params=params)

### Count of Referrals by Source
Count of referrals grouped by source for referrals with a referral date during the period. Returned as a pandas dataframe with columns `referral_source` and `referrals`.

In [ ]:
enrollment.referral_source_count(params=params).head()

### Most Common Referral Source
Creates the `referral_source_count` table and returns the first row as a tuple.

In [ ]:
enrollment.most_common_referral_source(params=params)

### Enrollment Conversion Rate by Referral Source
Count of referrals with a non-null enrollment_effective date divided by the count of referrals grouped by source for referrals with a referral date during the period. Returned as a pandas dataframe with columns `referral_source` and `enrollment_rate`.

In [ ]:
enrollment.referral_enrollment_rates_df(params=params).head()

### Source with the Highest Enrollment Conversion Rate
Creates the `referral_enrollment_rates_df` table and returns the first row as a tuple.

In [ ]:
enrollment.highest_enrollment_rate_referral_source(params=params)

## Demographics

In [ ]:
demographics = pu.Demographics()

### Number of Dual Enrolled Ppts
Count of ppts enrolled in the period with a medicare value of 1 and a medicaid value of 1.

In [ ]:
demographics.dual_count(params=params)

### Percent of Ppts that are Dual Enrolled 
`dual_count` divided by `census_during_period`.

In [ ]:
demographics.percent_dual(params=params)

### Number of Medicare Only Ppts
Count of ppts enrolled in the period with a medicare value of 1 and a medicaid value of 0.

In [ ]:
demographics.medicare_only_count(params=params)

### Percent of Ppts that are Medicare Only
`medicare_only_count` divided by `census_during_period`.

In [ ]:
demographics.percent_medicare_only(params=params)

### Number of medicaid Only Ppts
Count of ppts enrolled in the period with a medicare value of 0 and a medicaid value of 1.

In [ ]:
demographics.medicaid_only_count(params=params)

### Percent of Ppts that are medicaid Only

`medicaid_only_count` divided by `census_during_period`.

In [ ]:
demographics.percent_medicaid_only(params=params)

### Number of Private Pay Ppts
Count of ppts enrolled in the period with a medicare value of 0 and a medicaid value of 0.

In [ ]:
demographics.private_pay_count(params=params)

### Percent of Ppts that are Private Pay
`private_pay_count` divided by `census_during_period`.

In [ ]:
demographics.percent_private_pay(params=params)

### Average Age
Age is calculated as the difference between the end date and the ppt's date of birth.
Average of age for ppts enrolled in the period.

In [ ]:
demographics.avg_age(params=params)

### Number of Ppts Below Age 65
Count of ppts with an age below 65 as of the end date in the params.

In [ ]:
demographics.age_below_65(params=params)

### Percent of Ppts Below Age 65
`age_below_65` divided by `census_during_period`.

In [ ]:
demographics.percent_age_below_65(params=params)

### Number of Ppts Above Age 65
Difference between `census_during_period` and `age_below_65`.

In [ ]:
demographics.age_above_65(params=params)

### Percent Ppts with a Primary Language that is not English
Count of rows in demographic table where the language is 'English' divided by the count of ppts enrolled in the period.

In [ ]:
demographics.percent_primary_non_english(params=params)

### Percent of Ppts with a race that is not Caucasian/White
Count of rows in demographic table where the race is 'Caucasian/White' divided by the count of ppts enrolled in the period.

In [ ]:
demographics.percent_non_white(params=params)

### Number of Female Ppts
Count of ppts in the demographics table where the gender is 1 and the ppt is enrolled.

In [ ]:
demographics.female_count(params=params)

### Percent of Ppts that are Female
`female_count` divided by `census_during_period`

In [ ]:
demographics.percent_female(params=params)

### Number of Ppts with a Behavioral Health Diagnosis
Count of distinct ppts from dx table where the ICD10 code is in F2, F31, F32, F33, F4, or F6 and the ppt is enrolled during the period.

In [ ]:
demographics.behavorial_dx_count(params=params)

### Percent of Ppts with a Behavioral Health Diagnosis
`behavorial_dx_count` divided by `census_during_period`

In [ ]:
demographics.behavorial_dx_percent(params=params)

### Number of Ppts with a Dementia Related Diagnosis
Count of distinct ppts from dx table where the ICD10 code is in F01.50, F01.51, F02.80, F02.81, F03.90, F03.91, F10.27, F10.97, 
F13.27, F13.97, F18.17, F18.27, F19.27, F19.97, G31.09, G31.83, G30.00, G30.10, G30.08, G30.09 and the ppt is enrolled during the period.

In [ ]:
demographics.dementia_dx_count(params=params)

### Percent of Ppts with a Dementia Related Diagnosis
Count of distinct ppts from dx table where the ICD10 code is in the chronic condition list (see code for full list) and the ppt is enrolled during the period.

In [ ]:
demographics.at_least_one_chronic_condition_count(params=params)

### Count of Chronic Conditions per Ppt

Counts the number of distinct icd10 codes in the chronic condition list grouped by ppt enrolled during the period. Returns a dataframe with the columns `member_id` and `count`.

In [ ]:
demographics.chronic_condition_df(params=params).head()

### Number of Ppts with more than 6 Chronic Conditions
Creates the dataframe from `chronic_condition_df` and reutnrs the count of those rows with a count above 6.

In [ ]:
demographics.over_six_chronic_conditions_count(params=params)

### Percent of Ppts with more than 6 Chronic Conditions
`over_six_chronic_conditions_count` divided by `census_during_period`

In [ ]:
demographics.over_six_chronic_conditions_percent(params=params)

### Number of Ppts Living in the Community
Count of ppts enrolled during the period who do not have a custodial admission date during the period.

In [ ]:
demographics.living_in_community(params=params)

### Percent of Ppts Living in the Community
`living_in_community` divided by `census_during_period`.

In [ ]:
demographics.living_in_community_percent(params=params)

### Number of Ppts Indicated to Attend the Day Center
Count of distinct ppts from the center_days table enrolled during the period and the days column is not equal to 'PRN'.

In [ ]:
demographics.attending_day_center(params=params)

### Percent of Ppts Indicated to Attend the Day Center
`attending_day_center` divided by `census_during_period`.

In [ ]:
demographics.percent_attending_dc(params=params)

## Incidents
There are 5 incidents tables;
+ falls
+ med_errors
+ wounds
+ burns
+ infections

Some methods in the Incident class require an incident_table parameter to specify which type of incident the statistic is being calculated for.

In [ ]:
incidents = pu.Incidents()

### Incidents per 100 Member Months (Falls/Med Errors/Wounds/Burns/Infections) 
Count of incidents with a date_time_occured date during the period divided by the sum of the first of the month census for each month during the period multiplied by 100.

In [ ]:
incidents.incident_per_100MM(params=params, incident_table="falls")

In [ ]:
incidents.incident_per_100MM(params=params, incident_table="med_errors")

### Number of Incidents (Falls/Med Errors/Wounds/Burns/Infections) 
Count of incidents with a date_time_occured date during the period.

In [ ]:
incidents.total_incidents(params=params, incident_table="falls")

In [ ]:
incidents.total_incidents(params=incidents.last_quarter(), incident_table="falls")

### Number of Ppts with more than 1 Incident in Period (Falls/Med Errors/Wounds/Burns/Infections) 
Count of ppts with more than 1 incident during period.

In [ ]:
incidents.num_of_incident_repeaters(params=params, incident_table="falls")

### Number of Incidents by Ppts with more than 1 Incident in Period (Falls/Med Errors/Wounds/Burns/Infections) 
Sum of the count of incidents attributed to ppts with more than 1 incident in the period.

In [ ]:
incidents.incidents_by_repeaters(params=params, incident_table="falls")

### Number of Unique Ppts with an Incident in Period (Falls/Med Errors/Wounds/Burns/Infections) 
Count of distinct ppts with an incident in the period.

In [ ]:
incidents.ppts_w_incident(params=params, incident_table="falls")

### Percent of Incidents Attributed to Ppts with more than 1 Incident in the Period 
`incidents_by_repeaters` divided by `total_incidents`.

In [ ]:
incidents.percent_by_repeaters(params=params, incident_table="falls")

### Rate of Ppts with more than 1 Incident of Ppts with an Incident
`num_of_incident_repeaters` divided by `ppts_w_incident`.

In [ ]:
incidents.repeat_ppts_rate(params=params, incident_table="falls")

### Average Number of Incidents of Ppts with an Incident in the Period
Average number of incidents for participants who have had an incident in the period.

In [ ]:
incidents.incident_avg_value(params=params, incident_table="falls")

### Ppts with more Incidents than the Average During the Period
Ppts with more incidents during the period than the calculated average during the period.

In [ ]:
incidents.ppts_above_avg(params=params, incident_table="falls")

### Rate of Ppts with an Above Average Incident Count out of Ppts with an Incident
`ppts_above_avg` divided by `ppts_w_incident`.

In [ ]:
incidents.percent_of_ppts_over_avg(params=params, incident_table="falls")

### List of Ppts with More Than 1 Incident (Member Id, Incident Count)
Creates a list of member_id and incident count tuples for ppts with more than 1 incident in the period.

In [ ]:
incidents.ppts_w_multiple_incidents(params=params, incident_table="falls")

### Adjusted Incident Count (Falls/Med Errors/Wounds/Burns/Infections)
Count of incidents during period minus any incidents by ppts with an incidents count greater than the mean plus 3 standard deviations.

In [ ]:
incidents.adjusted_incident_count(params=params, incident_table="falls")

### Percent of Ppts without an Incident Ever
Count of ppts enrolled during period who do not have an incident divided by `census_during_period`.

In [ ]:
incidents.percent_without_incident_overall(params=params, incident_table="falls")

### Percent of Ppts without an Incident in the Period
Count of ppts enrolled during period who do not have an incident during the period divided by `census_during_period`.

In [ ]:
incidents.percent_without_incident_in_period(params=params, incident_table="falls")

### Number of Wounds Above Stage 1
Count of wounds with a date_time_occured during the period and an ulcer stage of Stage 2, Stage 3, Stage 4, or Unstageable.

In [ ]:
incidents.wounds_above_stage1(params=params)

### Number of Wounds that are Unstageable
Count of wounds with a date_time_occured during the period and an ulcer stage of Unstageable.

In [ ]:
incidents.unstageable_wounds_count(params=params)

### Number of Pressure Ulcers
Count of wounds with a date_time_occured during the period and a wound_type of Pressure Ulcer.

In [ ]:
incidents.pressure_ulcer_count(params=params)

### Pressure Ulcers per 100 Member Months
`pressure_ulcer_count` divided by `member_months` multiplied by 100.

In [ ]:
incidents.pressure_ulcer_per_100(params=params)

### Average Wound Healing Time
Average of the difference between the date_healed and the date_time_occured for wounds with a date_healed during the period.

In [ ]:
incidents.avg_wound_healing_time(params=params)

### Number of UTIs
Counts of infections during the period with an infection_type of UTI, URI, or Sepsis-Urinary.

In [ ]:
incidents.uti_count(params=params)

### UTIs per 100 Member Months
`uti_count` divided by `member_months` multiplied by 100.

In [ ]:
incidents.uti_per_100(params=params)

### Number of Sepsis Infections
Count of infections during the period where 'sepsis' is in the infection_type.

In [ ]:
incidents.sepsis_count(params=params)

### Sepsis Infection per 100 Member Months
`sepsis_count` divided by `member_months` multiplied by 100.

In [ ]:
incidents.sepsis_per_100(params=params)

### Number of Burns with Degree 3rd Degree of Higher
Count of burns during period with a burn_degree of Third or Fourth.

In [ ]:
incidents.third_degree_burn_count(params=params)

### Count of Burns Grouped by Degree
Count of burns during period grouped by degree.

In [ ]:
incidents.burn_degree_counts(params=params)

### Number of Incidents with Major Harm or Death (Falls/Med Errors)
Count of incidents during period where the severity is equal to Major Harm or Death.

In [ ]:
incidents.major_harm_or_death_count(params=params, incident_table="falls")

### Count of Med Error Responsibility by Type
The sum of each of the responsibility columns (responsibility_pharmacy, responsibility _clinic, responsibility _home_care, responsibility _facility) in the med_errors table for med_errors occurring during the period. Returns a pandas dataframe with the columns `responsibility` and `count`.

In [ ]:
incidents.med_errors_responsibility_counts(params=params)

### Most Common Med Error Responsibility Type
Creates the `med_errors_responsibility_counts` table and returns the first row as a tuple.

In [ ]:
incidents.most_common_med_errors_responsibility(params=params)

### Number of Burns with an RN Assessment as a Follow Up
Sum of the assessment_rn column from burns for burns occurring during the period.

In [ ]:
incidents.rn_assessment_following_burn_count(params=params)

### Number of Med Errors Related to a High Risk Medication
Finds descriptions that contain a high risk medication.

In [ ]:
incidents.high_risk_med_error_count(params=params)

### Percent of Incidents Resulting in Major Harm or Death (Falls/Med Errors)
`major_harm_or_death_count` divided by `num_incident` multiplied by 100.

In [ ]:
incidents.major_harm_percent(params=params, incident_table="falls")

### Adjusted Incidents per 100 Member Months (Falls/Med Errors/Wounds/Burns/Infections)
`adjusted_incident_count` divided by `member_months` 

In [ ]:
incidents.adjusted_per_100MM(params=params, incident_table="falls")

### Percent of Wound that are Unstageable
`unstageable_wounds_count` divided by `total_incidents` (with parameter of "wounds")

In [ ]:
incidents.unstageable_wound_percent(params=params)

### Percent of Burns with Degree 3rd Degree of Higher
`third_degree_burn_count` divided by `total_incidents` (with parameter of "burns")

In [ ]:
incidents.third_degree_burn_rate(params=params)

### Percent of Burns that have an RN Assessment as Follow Up
`rn_assessment_following_burn_count` divided by `total_incidents` (with parameter of "burns")

In [ ]:
incidents.rn_assessment_following_burn_percent(params=params)

## Utilization

There are two utilization tables (inpatient and er_only). There are also 6 views of the inpatient table;
+ acute
+ psych
+ nursing_home
+ skilled
+ respite
+ custodial

Some methods in the Utilization class require a utilization_table parameter to specify which type of utilization the statistic is being calculated for.

In [ ]:
utilization = pu.Utilization()

### Admissions (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Count of admissions with an admission_date in the period.

In [ ]:
utilization.admissions_count(params=params, utilization_table="acute")

### Discharges (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Count of admissions with a discharge_date in the period.

In [ ]:
utilization.discharges_count(params=params, utilization_table="acute")

### Average Length of Stay  (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Average los (length of stay) for admissions with a discharge_date in the period.

In [ ]:
utilization.alos(params=params, utilization_table="acute")

### ER to Acute Admission Rate
Sum of the er column from the acute view divided by the sum of that value and the count of er_only admissions during the period.

In [ ]:
utilization.er_to_inp_rate(params=params)

### Average Length of Stay for Acute Admissions from the ER
Average los for admissions in the acute with an er value of 1 and an admission_date during the period.

In [ ]:
utilization.alos_for_er_admissions(params=params)

### Length of Stay per 100 Member Months
Sum of los for admissions with an admission_date during the period divided by the sum of the first of the month census for each month during the period multiplied by 100.

In [ ]:
utilization.los_per_100mm(params=params, utilization_table="acute")

### Number of 30 Day Readmits
Count of admissions during the period with a day_since_last_admission value below 30.

In [ ]:
utilization.readmits_30day(params=params, utilization_table="acute")

### 30 Day Readmit Rate
`readmits_30_day` divided by `admissions_count`

In [ ]:
utilization.readmits_30day_rate(params=params, utilization_table="acute")

### Number of Ppts in a Utilization Type During the Period (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Count of admissions with a discharge date that is null or greater than the start date and an admission_date less than the end date.

In [ ]:
utilization.ppts_in_utl(params=params, utilization_table="acute")

### Ppts in Utilization Type per 100 Member Months  (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
`ppts_in_utl` divided by `member_months` multiplied by 100. 

In [ ]:
utilization.ppts_in_utl_per_100MM(params=params, utilization_table="acute")

### Percent of Ppts in a Utilization Type
`ppts_in_utl` divided by `census_during_period`.

In [ ]:
utilization.ppts_in_utl_percent(params=params, utilization_table="acute")

### Discharge 30 Days or Less before Death
Checks for an admission with a discharge date between the decease_date and the date 30 days prior, returns 1 if an admission exists, 0 otherwise.

In [ ]:
utilization.check_for_admission_30days_before_death(member_id=100122, deceased_date="2007-03-26")

### Utilization Related to Condition or Diagnosis
Searches through the acute, er_only, admitting_claims, and claims_detail tables for any dx where the condition or condition_abr appear. Returns a pandas dataframe.

In [ ]:
utilization.utilization_related_to_condition(params=params,
                                             condition="Chronic Obstructive Pulmonary Disease",
                                             condition_abr="COPD")

### LOS Over X During Period DataFrame
Finds admissions with a discharge_date during the period and a los values above the provided *x*.

In [ ]:
utilization.los_over_x_df(params=params, x=7, utilization_table="acute").head()

### Number of Admissions with a LOS Over X During Period
Count of admissions with a discharge_date during the period and a los values above the provided *x*.

In [ ]:
utilization.los_over_x_count(params=params, x=7, utilization_table="acute")

### Days Over X During Period DataFrame
Days are calculated by subtracting the admission date from either the discharge date or the end date. This function then returns the admissions during the period with days above *x*.

In [ ]:
utilization.days_over_x_df(params=params, x=7, utilization_table="acute").head()

### Number of Days for Utilization Type in Period
First the days for admissions that occur during the params is calculated. Next the days for admissions occurring before the period are calculated using the start date of the period as the admissions date. The sum of these two values are returned as the total utilization days during a given period.

In [ ]:
utilization.utilization_days(params=params, utilization_table="acute")

### Days for Utilization Type per 100 Member Months
`utilization_days` divided by `member_months`.

In [ ]:
utilization.days_per_100MM(params=params, utilization_table="acute")

### Admit Reasons by LOS
Finds admissions with a discharge_date during the period, orders them by los and then returns the *top_x* number of rows from that query as a pandas dataframe.

In [ ]:
utilization.top_admit_reason_by_los(params=params, top_x=10)

### Top 10 ER Users
Counts the numbers of visits with an admission_date during the period grouped by participants and returns the top 10 rows as a pandas dataframe.

In [ ]:
utilization.top_10_er_users(params=params)

### 30 Day Readmission DataFrame
Returns a pandas dataframe of any admissions with an admission_date during the period and a day_since_last_admission value less than or equal to 30.

In [ ]:
utilization.admissions_30day_readmit_df(params=params, utilization_table="acute").head()

### Admissions Resulting in a 30 Day Readmit DataFrame
Finds the admission prior to any admissions in the `acute_admissions_30day_readmit_df`. Returns these admissions as a pandas dataframe.

In [ ]:
utilization.admissions_resulting_in_30day_df(params=params, utilization_table="acute").head()

### Unique Ppts with an Admission (Acute/Psych/Nursing Home/Custodial/Respite/Skilled)
Returns a count of the distinct ppts with an admission during the period.

In [ ]:
utilization.unique_admissions_count(params=params, utilization_table="acute")

### Number of Weekend Admissions
Returns a count of the admissions during the period with a dow value of 'Saturday' or 'Sunday'

In [ ]:
utilization.weekend_admissions_count(params=params, utilization_table="acute")

### Percent of Admissions that Occur on the Weekend
`weekend_admissions_count` divided by `admissions_count` multiplied by 100.

In [ ]:
utilization.weekend_admission_percent(params=params, utilization_table="acute")

### Admissions per 100 Member Months
Sum of the count of admissions with an admission_date during the period divided by the sum of the first of the month census for each month during the period multiplied by 100.

In [ ]:
utilization.admissions_per_100MM(params=params, utilization_table="acute")

### Number of Nursing Facility Discharges to Custodial or Acute Hospital 
Finds all nursing_home admissions or discharges during the period. Merges any discharges that have a disposition of "Nursing home or rehabilitation facility" with all nursing_home admissions on the discharge_date matching the admission_date. Filter to only include those discharges with a non-null admission_date and a new admit_reason of "custodial" where a previous admit_reason was not custodial and count the discharges in this filtered group.

It then sums up any discharges to "Acute care hospital or psychiatric facility".

These two values are added together and returned. 

In [ ]:
utilization.nf_discharged_to_higher_loc(params=params)

### Percent of Nursing Facility Discharges that are to Custodial or Acute Hospital
`nf_discharged_to_higher_loc` divided by `discharges_count` for the nursing_home table multiplied by 100.

In [ ]:
utilization.percent_nf_discharged_to_higher_loc(params=params)

## Quality

In [ ]:
quality = pu.Quality()

### Ppts Who Need Pneumococcal 23 Vaccination DataFrame
Finds any ppts over 65 and enrolled during the period who does not have a record of receiving Pneumococcal 23. Returns a pandas dataframe of these ppts.

In [ ]:
quality.dataframe_query("SELECT * FROM ppts").columns

In [ ]:
quality.need_pneumo_23_df(params=params).head()

### Ppts Who Need PCV 13 Vaccination DataFrame
Find any ppts over 65 and enrolled during the period who does not have a record of receiving PCV 13. Returns a pandas dataframe of these ppts.

In [ ]:
quality.need_pcv_13_df(params=params).head()

### Ppts Who Need ONLY Pneumococcal 23 Vaccination DataFrame

Finds any ppts over 65 and enrolled during the period who does not have a record of receiving Pneumococcal 23 and has a record of receiving PCV 13. Returns a pandas dataframe of these ppts.

In [ ]:
quality.need_pneumo_23_only_df(params=params).head()

### Ppts Who Need ONLY PCV 13 Vaccination DataFrame
Finds any ppts over 65 and enrolled during the period who does not have a record of receiving PCV 13 and has a record of receiving Pneumococcal 23. Returns a pandas dataframe of these ppts.

In [ ]:
quality.need_pcv_13_only_df(params=params).head()

### Ppts Who Need BOTH Pneumococcal 23 & PCV 13 Vaccination DataFrame

Returns ppts who do not have either PCV 13 or Pneumococcal 23. If include_refused is True, will include participants who have previously refused the vaccination as still needing it, otherwise they are removed.

In [ ]:
quality.need_both_pneumo_vaccs_df(params=params).head()

### Number of Ppts with Pneumococcal Vaccination
Count of all enrolled ppts over the age of 65 who have at least one of the pneumo vaccinations.

In [ ]:
quality.has_pneumo_vacc_count(params=params)

### Number of Ppts who Refused Pneumococcal Vaccination

Count of all enrolled ppts over the age of 65 who have refused the vaccination and not received at least one of the pneumo vaccinations.

In [ ]:
quality.refused_pneumo_vacc_count(params=params)

### Pneumococcal Vaccination Rate
`has_pneumo_vacc_count` plus `refused_pneumo_vacc_count` divided by `age_above_65`.

In [ ]:
quality.pneumo_rate(params=params)

### Ppts who Refused Pneumococcal Vaccination DataFrame

Dataframe of all enrolled ppts over the age of 65 who have refused the vaccination and not received at least one of the pneumo vaccinations.

In [ ]:
quality.refused_pneumo_vacc_df(params=params).head()

### Ppts with Pneumococcal Vaccination DataFrame
Dataframe of all enrolled ppts over the age of 65 who have at least one of the pneumo vaccinations.

In [ ]:
quality.has_pneumo_vacc_df(params=params).head()

### Influenza Related Calculations
If the start date of the period is before April the start of the flu season is considered to be September 1st of the previous year. Other wise it will be the 1st of September in the same year as the parameter start.

This means the period of April through August will always have no results.

### Ppts Who Need to be Offered the Influzenza Vaccination DataFrame
Returns a dataframe of ppts enrolled during the period who do not have an influenza action (administer or refused) during the related flu season.

In [ ]:
quality.need_influenza_vacc_df(params=params).head()

### Number of Ppts with Influzenza Vaccination
Count of ppts who have an influenza vaccination (dose_status=1) during the related flu season.

In [ ]:
quality.has_influ_vacc_count(params=params)

### Ppts with Influzenza Vaccination DataFrame
Returns a dataframe of ppts enrolled during the period who have an influenza vaccination (dose_status=1) during the related flu season.

In [ ]:
quality.has_influ_vacc_df(params=params).head()

### Number of Ppts who Refused Influenza
Count of ppts who have refused the influenza vaccination (dose_status=0) during the related flu season and not subsequently received it.

In [ ]:
quality.refused_influ_vacc_count(params=params)

### Ppts who Refused Influenza DataFrame
Returns a dataframe of ppts enrolled during the period who have refused an influenza vaccination during the related flu season.

In [ ]:
quality.refused_influ_vacc_df(params=params).head()

### Influenza Vaccination Rate
The sum of ppts enrolled during the period who have either received or refused the influenza vaccination divided by the census during the period.

In [ ]:
quality.influ_rate(params=params)

### Mortality Rate
The number of deaths during the period divided by the census during the period.

In [ ]:
quality.mortality_rate(params=params)

### Mortality within 30 Days of Discharge Rate
The rate of deaths that occurred within 30 days of discharge from a hospital. This is deaths within 30 days of discharge divided by total deaths during the period.

In [ ]:
quality.mortality_within_30days_of_discharge_rate(params=params)

### Percent of Discharges that Result in Death within 30 Days
The rate of discharges that result in death within 30 days. This is deaths within 30 days of discharge divided by total hospital discharges during the period.

In [ ]:
quality.percent_of_discharges_with_mortality_in_30(params=params)

### Percent of Ppts without Acute Admission Since Enrollment
Percent of ppts who do not have a recorded acute hospital admissions.

In [ ]:
quality.no_hosp_admission_since_enrollment(params=params)

### Percent of Ppts without Acute Admission in the Last Year
Percent of ppts who do not have a recorded acute hospital admissions within the year prior to the provided end date.

In [ ]:
quality.no_hosp_admission_last_year(params=params)

### Average Days From Enrollment to First Custodial Admission or Average Days Until Nursing Facility Admission
The average of the difference between the first admission date to a custodial stay and the ppt enrollment date for ppts enrolled in the period.

In [ ]:
quality.avg_days_until_nf_admission(params=params)

## Team

Team functions take the functions listed in the modules above and create dataframe where each row is the team and the value of the function for that team.

In [ ]:
team = pu.Team()

### Admissions by Team

In [ ]:
team.admissions_by_team(params=params, utilization_table="acute")

### Utilization Days by Team

In [ ]:
team.days_by_team(params=params, utilization_table="acute")

### Discharges by Team

In [ ]:
team.discharges_by_team(params=params, utilization_table="acute")

### Average Length of Stay by Team

In [ ]:
team.alos_for_discharges_by_team(params=params, utilization_table="acute")

### 30 Day Readmissions by Team

In [ ]:
team.readmits_by_team(params=params, utilization_table="acute")

### Average Age by Team

In [ ]:
team.avg_age_by_team(params=params)

### Percent Non-English Primary Speakers by Team

In [ ]:
team.percent_primary_non_english_by_team(params=params)

### Average Years Enrolled in PACE by Team

In [ ]:
team.avg_years_enrolled_by_team(params=params)

### Number of Ppts in Custodial Care by Team

In [ ]:
team.ppts_in_custodial_by_team(params=params)

### Number of Ppts on Team

In [ ]:
team.ppts_on_team(params=params)

### Member Months during Period for Team

In [ ]:
team.team_member_months(params=params)

### Mortality Rate by Team

In [ ]:
team.mortality_by_team(params=("2017-01-01", "2019-10-01"))

### Deaths by Team

In [ ]:
team.mortality_by_team(params=params, total=True)

### Mortality within 30 Days of Discharge

In [ ]:
team.mortality_within_30days_of_discharge_by_team_df(params=params)

### Percent of Discharges with a Death within 30 Days by Team

In [ ]:
team.percent_of_discharges_with_mortality_in_30_by_team(params=params)

### Mortality within 30 Days of Discharge Rate by Team

In [ ]:
team.mortality_within_30days_of_discharge_rate_by_team(params=params)

### Percent of Ppts with No Acute Admission Since Enrollment by Team

In [ ]:
team.no_hosp_admission_since_enrollment_by_team(params=params)

### Pressure Ulcer Rate by Team

In [ ]:
team.pressure_ulcer_rate_by_team(params=params)

### Incidents by Team (Falls/Med Errors/Wounds/Burns/Infections)

In [ ]:
team.total_incidents_by_team(params=params, incident_table="falls")

### Incidents per Member by Team (Falls/Med Errors/Wounds/Burns/Infections)

In [ ]:
team.incidents_per_member_by_team(params=params, incident_table="falls")

### Number of Ppts with an Incident by Team (Falls/Med Errors/Wounds/Burns/Infections)

In [ ]:
team.ppts_w_incident_by_team(params=params, incident_table="falls")

### Create a Time/Value DataFrame for Plotting

In [ ]:
team.loop_plot_team_df(team.ppts_on_team, params=params, freq="MS", col_suffix="_census")

### Create a Time/Value DataFrame for Plotting

In [ ]:
team.loop_plot_team_df(team.ppts_w_incident_by_team,
                       params=params,
                       freq="QS",
                       additional_func_args = ["falls"],
                       col_suffix="_census")

## Agg

Defaults to the Agg SQLite database(V:\Databases\agg.db), contains a few tables with aggregate data by month or quarter. Mostly used for plotting on the dashboard

Add `_q` to any table below to return the data aggregated by quarter instead of month.

+ enrollment
+ demographics
+ falls
+ med_errors
+ wounds
+ burns
+ infections
+ utilization
+ quality
+ team_utl
+ team_info
+ team_incidents
+ center_enrollment

In [ ]:
agg = pu.Agg()

In [ ]:
agg.get_plot_df(table="enrollment", col="census", params=("2019-01-01", "2019-10-01"))

In [ ]:
agg.get_plot_df(table="enrollment_q", col="census", params=("2019-01-01", "2019-10-01"))

In [ ]:
agg.team_plot_df(table="team_info", col="ppts", params=("2019-01-01", "2019-10-01"))

In [ ]:
agg.team_plot_df(table="team_utl_q", col="custodial_ppts", params=("2019-01-01", "2019-10-01"))

# WIP(Beta) Classes

## Participant

In [ ]:
ppt = pu.Participant()

### Utilization Related to Ppt in Period

In [ ]:
ppt.utilization(params=("2018-09-01", "2019-09-30"), utilization_table="acute", member_id=100811)

### Get Ppt Name from Member ID

In [ ]:
ppt.name(member_id=100811)

## Center Classes
### CenterEnrollment & CenterDemographics

Working on completing these or moving to add a "center" parameter to all of the functions in the classes above.

In [ ]:
center_enrollment = pu.CenterEnrollment()

In [ ]:
center_enrollment.census_during_period(params=params, center="Providence")

In [ ]:
enrolled_addresses, disenrolled_addresses = center_enrollment.address_mapping_df(center="Providence")
enrolled_addresses.head()

In [ ]:
center_demographics = pu.CenterDemographics()

In [ ]:
print(center_demographics.percent_primary_non_english(params, center="Providence"))
print(center_demographics.percent_primary_non_english(params, center="Woonsocket"))
print(center_demographics.percent_primary_non_english(params, center="Westerly"))